
## Sentiment analysis using Neural Networks



In this project we will perform sentiment analysis using a few simple Neural Network based architectures.
For this problem we use the IMDB Large Movie Review Dataset. The dataset contains 25,000 highly polar movie reviews for both train and test dataset, each with 12,500 positive (greater than equal to 7/10 rating) and 12,500 negative reviews(less than equal to 4/10 rating). 

Use "https://keras.io/" for keras documentation. Please use Python 3. GPU is not required but it will help improve the training speed for each problem.



In [1]:
from os import listdir
import random
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Dense, Dropout, Reshape, Merge, BatchNormalization, TimeDistributed, Lambda, Activation, LSTM, Flatten, Convolution1D, GRU, MaxPooling1D
from keras.regularizers import l2
from keras.callbacks import Callback, ModelCheckpoint, EarlyStopping
from keras import backend as K
from keras.optimizers import SGD
from keras.optimizers import Adadelta
from keras.utils import np_utils
from keras.preprocessing import sequence
from keras import optimizers
import numpy as np

Using TensorFlow backend.


In [2]:


#we retrieve train and test file names

train_dir = "./aclImdb/train/"
test_dir = "./aclImdb/test/"
tr_review = [re_filename for re_filename in listdir(train_dir)]
te_review = [re_filename for re_filename in listdir(test_dir)]

#we initialize the train and test arrays

tr_X = []
tr_Y = []
te_X = []
te_Y = []

#we arrange the reviews into the train and test arrays 

for review_file in tr_review:
    f_review = open(train_dir+review_file, "r")
    str_review = f_review.readline()
    str_review = " ".join(str_review.split(' '))
    tr_X.append(str_review)
    y_truth = int (review_file.split('.')[0].split('_')[1])
    if y_truth>=7:
        tr_Y.append(1)
    else:
        tr_Y.append(0)
        
for review_file in te_review:
    f_review = open(test_dir+review_file, "r")
    str_review = f_review.readline()
    str_review = " ".join(str_review.split(' '))
    te_X.append(str_review)
    y_truth = int (review_file.split('.')[0].split('_')[1])
    if y_truth>=7:
        te_Y.append(1)
    else:
        te_Y.append(0)
        

We will now create the validation set from the train set

In [3]:
 
SEED = 2346
seed_counter = 0
while(1):

    shuffle_combine = list(zip(tr_X, tr_Y))
    random.seed(SEED+seed_counter)
    seed_counter+=1
    random.shuffle(shuffle_combine)

    tr_X, tr_Y = zip(*shuffle_combine)

    val_X = tr_X[:5000]
    val_Y = tr_Y[:5000]

    counter = 0
    for label in val_Y:
        counter+=label

    print (counter)
    print (seed_counter)
    if(counter>2400 and counter <2600):
        tr_X = tr_X[5000:]
        tr_Y = tr_Y[5000:]
        break

2446
1


In [4]:


print("Length of Train review set : " + str(len(tr_X)))
print("Length of Train label set : " + str(len(tr_Y)))
print("Length of Validation review set : " + str(len(val_X)))
print("Length of Validation label set : " + str(len(val_Y)))
print("Length of Test review set : " + str(len(te_X)))
print("Length of Test label set : " + str(len(te_Y)))
print("*****************************************")
print("Some sample Reviews Train sets and their labels")
print(tr_X[0][:150])
print(tr_Y[0])
print(tr_X[1][:150])
print(tr_Y[1])
print(tr_X[2][:150])
print(tr_Y[2])
print(tr_X[3][:150])
print(tr_Y[3])
print(tr_X[4][:150])
print(tr_Y[4])

Length of Train review set : 20000
Length of Train label set : 20000
Length of Validation review set : 5000
Length of Validation label set : 5000
Length of Test review set : 25000
Length of Test label set : 25000
*****************************************
Some sample Reviews Train sets and their labels
I don't know what the previous reviewer was watching but I guess that's what reviews are, personal taste. Missed in this movie was the depth, a very d
1
Many people here say that this show is for kids only. Hm, when I was a kid (approximately 7-9 years old) I watched this show first. It was disgusting 
0
This is the most elementary sort of traditional ghost story, not even enlivened to any great extent by the use of Irish locations. If the great M.R. J
0
I'll keep this short; thanks to Greg for helping me to put this succinctly: Captivity is about a guy who drugs a girl's drink, imprisons and tortures 
0
Although the recent re-telling of part of Homer's epic "Troy" with Brad Pitt was ente

In [5]:
#we collect all the reviews from train validation and test set to generate 
texts = []
texts += tr_X 
texts += te_X 
texts += val_X
len(texts)



#we clip the sentence length to first 250 words. 
MAX_SEQUENCE_LENGTH = 250

#length of vocab, Tokenizer will only use vocab_len most common words
vocab_len = 25000

#we tokenize the texts and convert all the words to tokens
tokenizer = Tokenizer(num_words=vocab_len)
tokenizer.fit_on_texts(texts)

token_tr_X = tokenizer.texts_to_sequences(tr_X)
token_te_X = tokenizer.texts_to_sequences(te_X)
token_val_X = tokenizer.texts_to_sequences(val_X)

#to ensure all reviews have the same length, we pad the smaller reviews with 0, 
#and cut the larger reviews to a max length 
#(we clip from the top, as the end of the reviews generally have a conclusion which provides better features)
x_train = sequence.pad_sequences(token_tr_X, maxlen=MAX_SEQUENCE_LENGTH)
x_test = sequence.pad_sequences(token_te_X, maxlen=MAX_SEQUENCE_LENGTH)
x_val = sequence.pad_sequences(token_val_X, maxlen=MAX_SEQUENCE_LENGTH)


#changes the labels to one-hot encoding
y_train = np_utils.to_categorical(tr_Y)
y_test = np_utils.to_categorical(te_Y)
y_val = np_utils.to_categorical(val_Y)


In [6]:
print('X_train shape:', x_train.shape)
print('X_test shape:', x_test.shape)
print('X_val shape:', x_val.shape)

print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)
print('y_val shape:', y_val.shape)


print("*****************************************")
print("Tokenized Reviews Train sets and their labels")
print(x_train[0][:20])
print(y_train[0])
print()
print(x_train[1][:20])
print(y_train[1])
print()
print(x_train[2][:20])
print(y_train[2])
print()
print(x_train[3][:20])
print(y_train[3])
print()
print(x_train[4][:20])
print(y_train[4])
print()

X_train shape: (20000, 250)
X_test shape: (25000, 250)
X_val shape: (5000, 250)
y_train shape: (20000, 2)
y_test shape: (25000, 2)
y_val shape: (5000, 2)
*****************************************
Tokenized Reviews Train sets and their labels
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[ 0.  1.]

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[ 1.  0.]

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[ 1.  0.]

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[ 1.  0.]

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[ 0.  1.]



********************************************

As you can see the reviews have now been transformed into indices to tokenized vocabulary and the labels have been converted to one-hot encoding. We can now go ahead and feed these sequences to Neural Network Models.

********************************************

# Part A

Building the first model 

Construct this sequential model using Keras :

![title](img/model1.jpg)

In [7]:
print('Build model...')
## implement model here
model = Sequential()
model.add(Embedding(vocab_len, 128, input_length=MAX_SEQUENCE_LENGTH))
model.add(Flatten())
model.add(Dense(200, activation = 'relu'))
model.add(Dense(2, activation = 'softmax'))

## compille it here according to instructions
model.compile(loss= 'categorical_crossentropy',optimizer= 'adam',metrics=[ 'accuracy' ])

model.summary()

print("Model Built")

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 250, 128)          3200000   
_________________________________________________________________
flatten_1 (Flatten)          (None, 32000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 200)               6400200   
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 402       
Total params: 9,600,602
Trainable params: 9,600,602
Non-trainable params: 0
_________________________________________________________________
Model Built


In [8]:
print('Train...')
model.fit(x_train, y_train,
          batch_size=32,
          epochs=4,
          validation_data=(x_val, y_val),
          verbose = 1,
         shuffle = True)


Train...
Train on 20000 samples, validate on 5000 samples
Epoch 1/4
20000/20000 [==============================] - 24s 1ms/step - loss: 0.3897 - acc: 0.8127 - val_loss: 0.2989 - val_acc: 0.8754
Epoch 2/4
20000/20000 [==============================] - 10s 495us/step - loss: 0.0360 - acc: 0.9886 - val_loss: 0.5035 - val_acc: 0.8494
Epoch 3/4
20000/20000 [==============================] - 10s 493us/step - loss: 0.0020 - acc: 0.9996 - val_loss: 0.6047 - val_acc: 0.8550
Epoch 4/4
20000/20000 [==============================] - 10s 497us/step - loss: 9.6887e-05 - acc: 1.0000 - val_loss: 0.6323 - val_acc: 0.8594


# Part B

Stacking Fully Connected Layers

Construct this sequential model using Keras :

![title](img/model2.jpg)

In [9]:
print('Build model...')

## implement model here

model = Sequential()
model.add(Embedding(vocab_len, 128, input_length=MAX_SEQUENCE_LENGTH))
model.add(Flatten())
model.add(Dense(200, activation = 'relu'))
model.add(Dense(200, activation = 'relu'))
model.add(Dense(2, activation = 'softmax'))

## compille it here according to instructions
model.compile(loss= 'categorical_crossentropy',optimizer= 'adam',metrics=[ 'accuracy' ])
model.summary()

print("Model Built")

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 250, 128)          3200000   
_________________________________________________________________
flatten_2 (Flatten)          (None, 32000)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 200)               6400200   
_________________________________________________________________
dense_4 (Dense)              (None, 200)               40200     
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 402       
Total params: 9,640,802
Trainable params: 9,640,802
Non-trainable params: 0
_________________________________________________________________
Model Built


In [10]:
print('Train...')
model.fit(x_train, y_train,
          batch_size=32,
          epochs=4,
          validation_data=(x_val, y_val),
          verbose = 1,
         shuffle = True)

Train...
Train on 20000 samples, validate on 5000 samples
Epoch 1/4
20000/20000 [==============================] - 11s 555us/step - loss: 0.4002 - acc: 0.8036 - val_loss: 0.2979 - val_acc: 0.8748
Epoch 2/4
20000/20000 [==============================] - 10s 512us/step - loss: 0.0426 - acc: 0.9847 - val_loss: 0.5193 - val_acc: 0.8428
Epoch 3/4
20000/20000 [==============================] - 10s 512us/step - loss: 0.0059 - acc: 0.9981 - val_loss: 0.7502 - val_acc: 0.8484
Epoch 4/4
20000/20000 [==============================] - 10s 511us/step - loss: 0.0123 - acc: 0.9957 - val_loss: 0.6448 - val_acc: 0.8422


# Part C

Using LSTMS based networks

Construct this sequential model using Keras :

![title](img/model3.jpg)

In [11]:
print('Build model...')

## implement model here

model = Sequential()
model.add(Embedding(vocab_len, 128, input_length=MAX_SEQUENCE_LENGTH))
model.add(LSTM(128))
model.add(Dense(128, activation = 'relu'))
model.add(Dense(2, activation = 'softmax'))

## compille it here according to instructions
model.compile(loss= 'categorical_crossentropy',optimizer= 'adam',metrics=[ 'accuracy' ])
model.summary()


print("Model Built")

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 250, 128)          3200000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_6 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_7 (Dense)              (None, 2)                 258       
Total params: 3,348,354
Trainable params: 3,348,354
Non-trainable params: 0
_________________________________________________________________
Model Built


In [12]:

print('Train...')
model.fit(x_train, y_train,
          batch_size=32,
          epochs=5,
          validation_data=(x_val, y_val),
          verbose = 1,
         shuffle = True)

Train...
Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 250s 13ms/step - loss: 0.4481 - acc: 0.7865 - val_loss: 0.3416 - val_acc: 0.8632
Epoch 2/5
20000/20000 [==============================] - 248s 12ms/step - loss: 0.2246 - acc: 0.9148 - val_loss: 0.5482 - val_acc: 0.8498
Epoch 3/5
20000/20000 [==============================] - 247s 12ms/step - loss: 0.1357 - acc: 0.9514 - val_loss: 0.3828 - val_acc: 0.8668
Epoch 4/5
20000/20000 [==============================] - 248s 12ms/step - loss: 0.0732 - acc: 0.9751 - val_loss: 0.5137 - val_acc: 0.8664
Epoch 5/5
20000/20000 [==============================] - 246s 12ms/step - loss: 0.0462 - acc: 0.9852 - val_loss: 0.5766 - val_acc: 0.8538


# Part D

Adding Pretrained Word Embeddings

Construct this sequential model using Keras :

Correction: The Embedding Layer Dimension (1st box) is 300, not 128.

![title](img/model4.jpg)

In [27]:
import codecs


#dimension of Glove Embeddings.
EMBEDDING_DIM = 300

word_index = tokenizer.word_index
print('Found %s unique tokens' % len(word_index))

#load glove embeddings
gembeddings_index = {}
with codecs.open('glove.42B.300d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        gembedding = np.asarray(values[1:], dtype='float32')
        gembeddings_index[word] = gembedding
#
f.close()
print('G Word embeddings:', len(gembeddings_index))

# nb_words contains the total length of vocab
nb_words = len(word_index) +1

#get glove embeddings for each word in tokenizer.
#g_word_embedding_matrix holds the embeddings dictionary
g_word_embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))

for word, i in word_index.items():
    gembedding_vector = gembeddings_index.get(word)
    if gembedding_vector is not None:
        g_word_embedding_matrix[i] = gembedding_vector
        
#total words in the tokenizer not in Embedding matrix
print('G Null word embeddings: %d' % np.sum(np.sum(g_word_embedding_matrix, axis=1) == 0))



Found 124252 unique tokens
G Word embeddings: 1917494
G Null word embeddings: 35772


In [28]:
print('Build model...')

## implement model here

model = Sequential()

## to use the glove embeddings, your embedding layer would take the vocab size as input dimension, 
## Glove embedding dimension as the output dimsion
## and you will provide the  embedding dictionary as the 'weights' parameter (!important) to the embedding layer.
model.add(Embedding(nb_words,EMBEDDING_DIM,weights=[g_word_embedding_matrix],input_length=MAX_SEQUENCE_LENGTH,trainable=False))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(128, activation = 'relu'))
model.add(Dense(2, activation = 'softmax'))

## compille it here according to instructions
model.compile(loss= 'categorical_crossentropy',optimizer= 'adam',metrics=[ 'accuracy' ])

model.summary()

print("Model Built")

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 250, 300)          37275900  
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               219648    
_________________________________________________________________
dense_8 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_9 (Dense)              (None, 2)                 258       
Total params: 37,512,318
Trainable params: 236,418
Non-trainable params: 37,275,900
_________________________________________________________________
Model Built


In [22]:
print('Train...')
model.fit(x_train, y_train,
          batch_size=32,
          epochs=5,
          validation_data=(x_val, y_val),
          verbose = 1,
         shuffle = True)

Train...
Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 279s 14ms/step - loss: 0.4662 - acc: 0.7704 - val_loss: 0.3304 - val_acc: 0.8548
Epoch 2/5
20000/20000 [==============================] - 278s 14ms/step - loss: 0.3050 - acc: 0.8706 - val_loss: 0.2941 - val_acc: 0.8734
Epoch 3/5
20000/20000 [==============================] - 278s 14ms/step - loss: 0.2733 - acc: 0.8868 - val_loss: 0.2774 - val_acc: 0.8832
Epoch 4/5
20000/20000 [==============================] - 273s 14ms/step - loss: 0.2440 - acc: 0.9011 - val_loss: 0.2745 - val_acc: 0.8850
Epoch 5/5
20000/20000 [==============================] - 274s 14ms/step - loss: 0.2230 - acc: 0.9103 - val_loss: 0.2693 - val_acc: 0.8922


# Part E

Using Convolutional Networks

Construct the model, shown below. Use the same loss functions and optimizers as before

Correction: The Embedding Layer Dimension (1st box) is 300, not 128.

![title](img/model6.jpg)

In [29]:
print('Build model...')

## implement model here

model = Sequential()
model.add(Embedding(nb_words,EMBEDDING_DIM,weights=[g_word_embedding_matrix],input_length=MAX_SEQUENCE_LENGTH,trainable=False))
model.add(Convolution1D(128,3))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Convolution1D(64,3))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Convolution1D(32,3))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(256))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Dense(2, activation = 'softmax'))



## compille it here according to instructions
model.compile(loss= 'categorical_crossentropy',optimizer= 'adam',metrics=[ 'accuracy' ])

model.summary()

print("Model Built")

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 250, 300)          37275900  
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 248, 128)          115328    
_________________________________________________________________
dropout_1 (Dropout)          (None, 248, 128)          0         
_________________________________________________________________
activation_1 (Activation)    (None, 248, 128)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 246, 64)           24640     
_________________________________________________________________
dropout_2 (Dropout)          (None, 246, 64)           0         
_________________________________________________________________
activation_2 (Activation)    (None, 246, 64)           0     

In [30]:

print('Train...')
model.fit(x_train, y_train,
          batch_size=32,
          epochs=5,
          validation_data=(x_val, y_val),
          verbose = 1,
         shuffle = True)

Train...
Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 11s 544us/step - loss: 0.4881 - acc: 0.7393 - val_loss: 0.3430 - val_acc: 0.8608
Epoch 2/5
20000/20000 [==============================] - 10s 482us/step - loss: 0.3082 - acc: 0.8737 - val_loss: 0.2990 - val_acc: 0.8768
Epoch 3/5
20000/20000 [==============================] - 10s 490us/step - loss: 0.2660 - acc: 0.8896 - val_loss: 0.2955 - val_acc: 0.8832
Epoch 4/5
20000/20000 [==============================] - 10s 492us/step - loss: 0.2240 - acc: 0.9096 - val_loss: 0.2912 - val_acc: 0.8782
Epoch 5/5
20000/20000 [==============================] - 10s 490us/step - loss: 0.1909 - acc: 0.9220 - val_loss: 0.3033 - val_acc: 0.8760


In [31]:
scores = model.evaluate(x_test, y_test, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

25000/25000 [==============================] - 4s 168us/step
Accuracy: 88.93%


# Part F

Create best model, use Validation score to judge the best model and check accuracy on test set


In [32]:
print('Build model...')

## implement model here
model = Sequential()
model.add(Embedding(nb_words,EMBEDDING_DIM,weights=[g_word_embedding_matrix],input_length=MAX_SEQUENCE_LENGTH,trainable=False))
model.add(Convolution1D(128,3))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Convolution1D(64,3))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Convolution1D(32,3))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(256))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Dense(128))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Dense(64))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Dense(2, activation = 'softmax'))

## compille it here according to instructions
model.compile(loss= 'categorical_crossentropy',optimizer= 'adam',metrics=[ 'accuracy' ])

model.summary()

print("Model Built")

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 250, 300)          37275900  
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 248, 128)          115328    
_________________________________________________________________
dropout_5 (Dropout)          (None, 248, 128)          0         
_________________________________________________________________
activation_5 (Activation)    (None, 248, 128)          0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 246, 64)           24640     
_________________________________________________________________
dropout_6 (Dropout)          (None, 246, 64)           0         
_________________________________________________________________
activation_6 (Activation)    (None, 246, 64)           0     

You can keep saving models with different names in model_name, 

so you can retrieve their weights again for testing, you dont have to retrain 
(You would have to initialize the model definition again).

In [33]:
wt_dir = "./weights/"
model_name = 'model_4'
early_stopping =EarlyStopping(monitor='val_acc', patience=2)
bst_model_path = wt_dir + model_name + '.h5'
model_checkpoint = ModelCheckpoint(bst_model_path, monitor='val_acc', save_best_only=True, save_weights_only=True)

print('Train...')
model.fit(x_train, y_train,
          batch_size=32,
          epochs=5,
          validation_data=(x_val, y_val),
          verbose = 1,
         shuffle = True,
         callbacks=[early_stopping, model_checkpoint])

model.save(model_name + '.h5')

model.save_weights(bst_model_path)

Train...
Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 14s 676us/step - loss: 0.4976 - acc: 0.7306 - val_loss: 0.3868 - val_acc: 0.8416
Epoch 2/5
20000/20000 [==============================] - 11s 564us/step - loss: 0.3199 - acc: 0.8700 - val_loss: 0.3340 - val_acc: 0.8634
Epoch 3/5
20000/20000 [==============================] - 11s 555us/step - loss: 0.2773 - acc: 0.8896 - val_loss: 0.2923 - val_acc: 0.8768
Epoch 4/5
20000/20000 [==============================] - 11s 546us/step - loss: 0.2386 - acc: 0.9047 - val_loss: 0.2844 - val_acc: 0.8830
Epoch 5/5
20000/20000 [==============================] - 10s 523us/step - loss: 0.2082 - acc: 0.9159 - val_loss: 0.2904 - val_acc: 0.8822


If you plan on using Ensemble averaging, feel free to edit the code below or add multiple models.

Make sure they get saved and can be retrieved when executing serially.

In [47]:
from keras.models import load_model
model=load_model('model_3.h5')
model.load_weights('./weights/model_3.h5')

scores = model.evaluate(x_test, y_test, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))



25000/25000 [==============================] - 4s 174us/step
Accuracy: 89.51%
